Text Classification (Sentiment Analysis) Using Bayes Rule 

In this task we wil implement a Naive Bayes Multinomial classifier using  bag of words model for the classification of text (movie reviews) into different categories..


In [2]:
import re
import math
%pylab inline
import scipy.stats
from collections import defaultdict
from collections import Counter

Populating the interactive namespace from numpy and matplotlib


In [3]:
def parse_string(string): # Parse a read string into respective tokens

    like=(re.sub("<.*?>", "",string[0]))
    tokens=re.findall(r'[\w+\'\w+]|\w+',like)
    return tokens

In [4]:
def parse_file(filename): # Parse a given file
    
    with open(filename,'r') as rfile:
        rstring=rfile.read()
    
    return rstring

In [5]:
def file_to_strings(X):

    rX=np.zeros((X.shape[0],1),dtype=object)
    for ei in range(X.shape[0]):
        rX[ei,0]=parse_file(X[ei][0])
        
    return rX

In [6]:
from collections import defaultdict

In [20]:
class NaiveBayes:
    ''' Implements the Naive Bayes For Text Classification... '''
    def __init__(self, classes):
        self.classes=classes
    def prior(self,lables):
        prior=[]
        classesFrequency=[]
        length=len(self.classes)
        for ind in range(length):
            classesFrequency.append(list(lables).count(self.classes[ind]))
            prior.append(float(classesFrequency[ind])/len(lables))
        return prior
    
    def splitClasses(self,X,Y):
        splitClass=[]
        for l in range(len(self.classes)):
            index=(Y==self.classes[l])
            list=X[index]
            splitClass.append(list)
        return splitClass
    
    def train(self, trainData, Y):
        # now go and train a model for each class...
        #Creating vocabulary of training data....
        posVocabulary=[]
        negVocabulary=[]
        feature,length,h=np.shape(trainData)
        for index in range(length):
            token=parse_string(trainData[0][index])
            negVocabulary.append(token)
            token=parse_string(trainData[1][index])
            posVocabulary.append(token)
        
        positive=np.concatenate(posVocabulary,axis=0)
        negative=np.concatenate(negVocabulary,axis=0)

        positiveUnique=np.unique(positive)
        negativeUnique=np.unique(negative)

        Vocalbulary=[negative,positive]
        uniqueVocalbulary=[negativeUnique,positiveUnique]
        vocalbularyOrignalLengths=[len(negative),len(positive)]
        vocalbularyUniqueLengths=[len(negativeUnique),len(positiveUnique)]
        return Vocalbulary,uniqueVocalbulary,vocalbularyOrignalLengths,vocalbularyUniqueLengths
    
    def test(self,testData,prior,v,uv,vol,vul):
            
        nexamples, nfeatures=testData.shape
        pclasses=[0]*nexamples
        # your code go here...
        neg = Counter(v[0])
        pos = Counter(v[1])
        print classes
        print "---------------Dictionary Done----------------"
        for inx in range(nexamples):#picking up file one by one
            token=parse_string(testData[inx])#parsing the picked file
            prob=[]
            for ine in range(len(self.classes)): #loop for types of reviews(first neg,then positive)
                likelyHood=0
                for ind in range(len(token)): #loop for each token of the file one by one
                    if (ine==0):
                        key=token[ind]
                        frequency=neg.get(key,0)
                    else:
                        key=token[ind]
                        frequency=pos.get(key,0)
                    num=frequency+1    
                    denom=vol[ine]+np.sum(vul)
                    likelyhood=float(num)/denom
                    lh=np.log(likelyhood)
                    likelyHood=lh+likelyHood 
                pr=np.log(prior[ine])
                probablity=pr+likelyHood 
                prob.append(probablity) 
            maxi=prob.index(max(prob))
            pclasses[inx]=self.classes[maxi] 
        return np.array(pclasses)
    

In [8]:
import pandas as pd
import tools as t

In [9]:
tdir='./data1/imdb1/' # training dir...
#load data, get list of files for each class...
posfiles=t.get_files(tdir+'/pos','*',withpath=True)
negfiles=t.get_files(tdir+'/neg','*',withpath=True)
pos=len(posfiles)
neg=len(negfiles)
print 'Number of positive reviews = ', pos
print 'Number of negative reviews = ', neg

Number of positive reviews =  1000
Number of negative reviews =  1000


In [10]:
#generate training and testing data...
plabels=['pos']*len(posfiles)
nlabels=['neg']*len(posfiles)
labels=np.concatenate((plabels,nlabels)) # concatenate the +ve and -ve labels
tX=np.concatenate((posfiles,negfiles)).reshape((len(posfiles)+len(negfiles),1))
print "Training data Dimensions =", tX.shape," Training labels dimensions=", labels.shape

Training data Dimensions = (2000L, 1L)  Training labels dimensions= (2000L,)


In [11]:
X=file_to_strings(tX) # read files and convert each file into set of strings and return an numpy array
#Split the data into two halves training and test set...
traindata,trainlabels,testdata,testlabels=t.split_data(tX,labels)
#Find the classes to train
classes=np.unique(labels)

In [14]:
#Now build a Naive Bayes classifir and test it...
print '[Info] training a classifier for following classes {}, {}'.format(classes[0],classes[1])
nb=NaiveBayes(classes)
testData=file_to_strings(testdata)
trainData=file_to_strings(traindata)
prior=nb.prior(labels)
split=nb.splitClasses(trainData,trainlabels)
v,uv,vol,vul=nb.train(split,trainlabels)
print "-------------Test--------------"
pclasses=nb.test(testData,prior,v,uv,vol,vul)
acc=np.sum(pclasses==testlabels)/float(testlabels.shape[0])
print "------------------Print ACC---------------------"
print "[Info] Accuracy = {}".format(acc)

[Info] training a classifier for following classes neg, pos
-------------Test--------------
['neg' 'pos']
---------------Dictionary Done----------------
------------------Print ACC---------------------
[Info] Accuracy = 0.836666666667


#Cross Validation

Now lets throw our methods to winds of different folds and measure their accuracy...

In [203]:
#Now lets generate n-fold training and testing data...

nfolds=10
folds=t.generate_folds(X,labels,nfolds) # generate folds for 
for k in arange(len(folds)):
    print folds[k][0].shape, folds[k][2].shape

Generating CV data for 2 classes
(1800L, 1L) (200L, 1L)
(1800L, 1L) (200L, 1L)
(1800L, 1L) (200L, 1L)
(1800L, 1L) (200L, 1L)
(1800L, 1L) (200L, 1L)
(1800L, 1L) (200L, 1L)
(1800L, 1L) (200L, 1L)
(1800L, 1L) (200L, 1L)
(1800L, 1L) (200L, 1L)
(1800L, 1L) (200L, 1L)


In [206]:
totacc=[]
#train a classifier for each fold...
classes=np.unique(labels)

for k in range(nfolds):
    nb=NaiveBayes(classes)
    prior=nb.prior(labels)
    traindata=folds[k][0]
    trainlabels=folds[k][1]
    testdata=folds[k][2]
    testlabels=folds[k][3]
    
    #Lets first train the classifier
    split=nb.splitClasses(traindata,trainlabels)
    v,uv,vol,vul=nb.train(split,trainlabels)
    
    #Lets test the classifier
    pclasses= nb.test(testdata,prior,v,uv,vol,vul)
    
    acc=np.sum(pclasses==testlabels)/float(testlabels.shape[0])
    print "[Info] Fold {} Accuracy = {}".format(k+1, acc)    
    
    totacc.append(acc)

print totacc
print '[Info] Mean Accuracy =', np.mean(totacc)

['neg' 'pos']
---------------Dictionary Done----------------
[Info] Fold 1 Accuracy = 0.795
['neg' 'pos']
---------------Dictionary Done----------------
[Info] Fold 2 Accuracy = 0.82
['neg' 'pos']
---------------Dictionary Done----------------
[Info] Fold 3 Accuracy = 0.785
['neg' 'pos']
---------------Dictionary Done----------------
[Info] Fold 4 Accuracy = 0.815
['neg' 'pos']
---------------Dictionary Done----------------
[Info] Fold 5 Accuracy = 0.825
['neg' 'pos']
---------------Dictionary Done----------------
[Info] Fold 6 Accuracy = 0.775
['neg' 'pos']
---------------Dictionary Done----------------
[Info] Fold 7 Accuracy = 0.81
['neg' 'pos']
---------------Dictionary Done----------------
[Info] Fold 8 Accuracy = 0.805
['neg' 'pos']
---------------Dictionary Done----------------
[Info] Fold 9 Accuracy = 0.825
['neg' 'pos']
---------------Dictionary Done----------------
[Info] Fold 10 Accuracy = 0.83
[0.79500000000000004, 0.81999999999999995, 0.78500000000000003, 0.8149999999999999

# Excellent, now its time to dive into deep waters of Kaggle.


In [9]:
# read the data-set
train=pd.read_csv('./kaggle/labeledTrainData.tsv',sep='\t')

In [10]:
train.describe()

,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [11]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [12]:
Yt=train['sentiment']
Xt=train['review']
Xt=np.array(Xt)
Yt=np.array(Yt)
print Xt.shape

(25000L,)


In [13]:
#read test set...
test=pd.read_csv('./kaggle/testData.tsv',sep='\t')

In [14]:
test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


####Training Time 

In [15]:
# Let's split the training data into two halves and test our accuracy...
traindata,trainlabels,testdata,testlabels=t.split_data(Xt.reshape((Xt.shape[0],1)),Yt)
classes=np.unique(trainlabels)

In [21]:
# Now lets go and train the model and see its performance...
print '[Info] training a classifier for following classes {}, {}'.format(classes[0],classes[1])
nb=NaiveBayes(classes)
prior=nb.prior(Yt)
split=nb.splitClasses(traindata,trainlabels)
l,lu,ol,ul=nb.train(split,trainlabels)
pclasses=nb.test(testdata,prior,l,lu,ol,ul)
acc=np.sum(pclasses==testlabels)/float(testlabels.shape[0])
print "[Info] Accuracy = {}".format(acc) 

[Info] training a classifier for following classes 0, 1
[0 1]
---------------Dictionary Done----------------
[Info] Accuracy = 0.615866666667


#### Cross-Validation Time...

In [38]:
#Split the training data into 10 folds and test classifiers performance...

nfolds=10
folds=t.generate_folds(Xt.reshape((Xt.shape[0],1)),Yt,nfolds) # generate folds for 
for k in arange(len(folds)):
    print folds[k][0].shape, folds[k][2].shape

Generating CV data for 2 classes
(22500L, 1L) (2500L, 1L)
(22500L, 1L) (2500L, 1L)
(22500L, 1L) (2500L, 1L)
(22500L, 1L) (2500L, 1L)
(22500L, 1L) (2500L, 1L)
(22500L, 1L) (2500L, 1L)
(22500L, 1L) (2500L, 1L)
(22500L, 1L) (2500L, 1L)
(22500L, 1L) (2500L, 1L)
(22500L, 1L) (2500L, 1L)


In [39]:
# As it takes time, so becareful it can cause your machine into red hot oven
totacc=[]
classes=np.unique(Yt)

for k in range(nfolds):
    nb=NaiveBayes(classes)
    prior=nb.prior(Yt)
    traindata=folds[k][0]
    trainlabels=folds[k][1]
    
    #Lets first train the classifier
    split=nb.splitClasses(traindata,trainlabels)
    l,lu,ol,ul=nb.train(split,trainlabels)
    
    testdata=folds[k][2]
    testlabels=folds[k][3]
    
    #Lets test the classifier
    pclasses= nb.test(testdata,prior,l,lu,ol,ul)
    
    acc=np.sum(pclasses==testlabels)/float(testlabels.shape[0])
    print "[Info] Fold {} Accuracy = {}".format(k+1, acc)    
    
    totacc.append(acc)

print totacc
print '[Info] Mean Accuracy =', np.mean(totacc)

[0 1]
---------------Dictionary Done----------------
[Info] Fold 1 Accuracy = 0.8508
[0 1]
---------------Dictionary Done----------------
[Info] Fold 2 Accuracy = 0.8392
[0 1]
---------------Dictionary Done----------------
[Info] Fold 3 Accuracy = 0.85
[0 1]
---------------Dictionary Done----------------
[Info] Fold 4 Accuracy = 0.8408
[0 1]
---------------Dictionary Done----------------
[Info] Fold 5 Accuracy = 0.8664
[0 1]
---------------Dictionary Done----------------
[Info] Fold 6 Accuracy = 0.8476
[0 1]
---------------Dictionary Done----------------
[Info] Fold 7 Accuracy = 0.8632
[0 1]
---------------Dictionary Done----------------
[Info] Fold 8 Accuracy = 0.8496
[0 1]
---------------Dictionary Done----------------
[Info] Fold 9 Accuracy = 0.8432
[0 1]
---------------Dictionary Done----------------
[Info] Fold 10 Accuracy = 0.8396
[0.8508, 0.83919999999999995, 0.84999999999999998, 0.84079999999999999, 0.86639999999999995, 0.84760000000000002, 0.86319999999999997, 0.84960000000000

# Now let's train on the complete dataset and test on the provided test set...

In [30]:
classes= np.unique(Yt)
print 'Training a Classifier on Full training set with classes =', classes
nb=NaiveBayes(classes)
prior=nb.prior(Yt)
split=nb.splitClasses(Xt.reshape(Xt.shape[0],1),Yt)
print np.shape(split)
l,lu,ol,ul=nb.train(split,Yt)
print "------------Done--------------"

Training a Classifier on Full training set with classes = [0 1]
(2L, 12500L, 1L)
------------Done--------------


In [31]:
#Get the test data...
print "------------Start---------------"
Xtest=test['review']
Xtest=np.array(Xtest.reshape((Xtest.shape[0],1)))
#test the classifier on the provided test set...
pclasses=nb.test(Xtest,prior,l,lu,ol,ul)
print "------------Done--------------"

------------Start---------------
[0 1]
---------------Dictionary Done----------------
------------Done--------------


In [32]:
#write the result in the kaggle's required format
output = pd.DataFrame( data={"id":test["id"], "sentiment":pclasses} )

# Use pandas to write the comma-separated output file
output.to_csv( "./kaggle/Naive_bays_Bag_of_Words_model.csv", index=False, quoting=3 )

# Time to Upload the prediction to Kaggle...

Now upload the result on the Kaggle and see your ranking and score. Using this simple method you can have an accuracy of around 0.80960.

#Improvement by Excluding Stop Words...

You can improve your score further by excluding the commonly occuring words (also known as stop words) in the English language.



In [36]:
#read and create a set of stop 
stopwords=set(t.read_txt_file('./data1/english.stop'))
#print stopwords
#--------------------Training-----------------------------
classes= np.unique(Yt)
print 'Training a Classifier on Full training set with classes =', classes
nb=NaiveBayes(classes)
prior=nb.prior(Yt)
split=nb.splitClasses(Xt.reshape(Xt.shape[0],1),Yt)
print np.shape(split)
l,lu,ol,ul=nb.train(split,Yt)
li.append( [item for item in l[0] if item not in stopwords])
li.append([item for item in l[1] if item not in stopwords])
print "----------------Stop Words Removed-------------"
print "------------Done--------------"
#--------------------Testing-----------------------------
#Get the test data...
print "------------Start---------------"
Xtest=test['review']
Xtest=np.array(Xtest.reshape((Xtest.shape[0],1)))
#test the classifier on the provided test set...
pclasses=nb.test(Xtest,prior,li,lu,ol,ul)
print "------------Done--------------"

Training a Classifier on Full training set with classes = [0 1]
(2L, 12500L, 1L)
----------------Stop Words Removed-------------
------------Done--------------
------------Start---------------
[0 1]
---------------Dictionary Done----------------
------------Done--------------


In [37]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":pclasses} )

# Use pandas to write the comma-separated output file
output.to_csv( "./kaggle/Naive_bays_Bag_of_Words_model.csv", index=False, quoting=3 )